# Probability distributions in HIC experiments

We study the relationship between two contigs in a HIC experiments - contig 1 and contig 2, with size $S_1$ and $S_2$, respectively. Our goal is to find out the probability distribution for the number of links between these two contigs, given their distance $D$. We can then compare against the observed number of links to try to infer if the two contigs are linked or not.

This distribution is used in the `partition` procedure in [ALLHIC](https://github.com/tanghaibao/allhic). If the two contigs are considered close enough, i.e. $Prob(D < threshold) > 0.5$, then we infer the two contigs to be linked. This link graph is then partitioned and distinct partition are then solved independently.

We use the following notations:
- $S_1$, $S_2$: size of contigs 1 and 2.
- $x_1$, $x_2$: start coordinates of contigs 1 and 2. Without loss of generality, we let $x_1 = 0$ and $x_2 > S_2$.
- $D$: inner distance between contigs 1 and 2. We require $D > 0$. By definition, we have $x_1 + S_1 + D = x_2$.

We build our models based on continuous distributions due to the large size of genomic positions. For each paired reads, we first sample the starting position $x$ uniformly on contig 1, that follows a uniform distribution:

$$f(x) = \begin{cases}
            \frac{1}{S_1}, & 0 \leq x \leq S_1 \\
            0, & \text{otherwise}
         \end{cases}
$$

We then consider the length $y$ of this link, that follows a [Pareto distribution](https://en.wikipedia.org/wiki/Pareto_distribution):

$$g(y) = \begin{cases}
            \frac{{\alpha} y_m^{\alpha}}{y^{\alpha + 1}}, & y \geq y_m \\
            0, & y < y_m
         \end{cases}
$$

Where $y_m$ is the smallest link length in the dataset. As a side note, we can use the set of intra-contig links to infer the MLE $\hat{\alpha}$, using the formula below.

$$\hat{\alpha} = \frac{n}{\sum_{i=1}^{n} \ln{y_i} - \ln{y_m}}
$$

Now let's come back to the problem of trying to infer the ending position $z$ of the read pair, which is the sum of two random variable $Z = X + Y$. Then $z$ is a convolution of two probability distribution - uniform and Pareto.

$$h(z) = \int_{0}^{S_1} f(x) \cdot g(z - x) dx
$$

In order to make $g(z - x)$ nonzero, we need $z - x \geq y_m$, borrowing idea from [here](https://math.stackexchange.com/questions/357672/density-of-sum-of-two-uniform-random-variables-0-1). Therefore, we have $x \leq z - y_m$, also $0 \leq x \leq S_1$. By comparing $z - y_m$ and $S_1$, we then break this down into two cases:

$$h(z) = \begin{cases}
            \frac{{\alpha} y_m^{\alpha}}{S_1} \int_{0}^{z - y_m} \frac{1}{(z - x)^{\alpha + 1}} dx, & z \leq S_1 + y_m \\
            \frac{{\alpha} y_m^{\alpha}}{S_1} \int_{0}^{S_1} \frac{1}{(z - x)^{\alpha + 1}} dx, & z > S_1 + y_m
         \end{cases}
$$

We are mostly interested in the second case, where the ending position occurs after $S_1$, i.e. beyond the extent of contig 1, so that it becomes a inter-contig link, rather than intra-contig link. 